In [33]:
import pandas as pd 
import numpy as np
from numpy import sqrt 
import array
from pymysql import NULL
import math

#Reading user file:
u_cols =  ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols, encoding='latin-1')
# đọc file csv và trả về 1 dataframe. Mặc định hàm này sẽ phân biệt các trường của file csv theo dấu phẩy. 
# sep: thay đổi dấu ngăn cách giữa các cột thành |
# encoding: chỉ định encoding của file đọc vào. Mặc định là utf-8. - > chuyển thành latin-1
#name : danh sacshteen cột để sử dụng
n_users = users.shape[0]
print ('Number of users:', n_users)
# users.head() #uncomment this to see some few examples

#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating']

ratings_base = pd.read_csv('ml-100k/ua.base', sep='\t', names=r_cols)
ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names=r_cols)

rate_train = ratings_base.to_numpy() # chuyển dữ liệu thành mảng 
rate_test = ratings_test.to_numpy()

print ('Number of traing rates:', rate_train.shape[0]) #in ra số lượng phần tử của mảng 
print ('Number of test rates:', rate_test.shape[0])

#i_cols = ['song id', 'song name' ,'singer','url', 'musician', 'Viet Nam', 'nhac tre', 'tru tinh', 'Remix', 'rap', 'tien chien','nhac trinh', 'rock việt', 'Cách Mạng', 'ÂU MỸ', 'Pop', 'Rock', 'Châu Á', 'Nhạc hàn', ' Nhạc Hoa', 'Nhạc Nhật','Nhạc Thái','Khác','Thiếu nhi', 'Không lời'] 
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western','hihi']


Number of users: 943
Number of traing rates: 90570
Number of test rates: 9430


In [34]:
items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols)

In [35]:
items.columns

Index(['movie id', 'movie title', 'release date', 'video release date',
       'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Children's',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western', 'hihi'],
      dtype='object')

In [36]:
for col in items.columns:
    if items[col].isnull().values.any():
        print(col)

release date
video release date
IMDb URL
hihi


In [37]:
items[items[].isnull().values]

SyntaxError: invalid syntax (<ipython-input-37-941aa6ecc40a>, line 1)

In [44]:

import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 

# data file 
r_cols = ['user_id', 'item_id', 'rating']
ratings = pd.read_csv('ml-100k/ex.dat', sep = ' ', names = r_cols, encoding='latin-1')
Y_data = ratings.to_numpy()

class CF(object):
    """docstring for CF"""
    def __init__(self, Y_data, k, dist_func = cosine_similarity, uuCF = 1):
        self.uuCF = uuCF # user-user (1) or item-item (0) CF
        self.Y_data = Y_data if uuCF else Y_data[:, [1, 0, 2]]
        self.k = k
        self.dist_func = dist_func
        self.Ybar_data = None
        # number of users and items. Remember to add 1 since id starts from 0
        self.n_users = int(np.max(self.Y_data[:, 0])) + 1 
        self.n_items = int(np.max(self.Y_data[:, 1])) + 1
    
    def add(self, new_data):
        """
        Update Y_data matrix when new ratings come.
        For simplicity, suppose that there is no new user or item.
        """
        self.Y_data = np.concatenate((self.Y_data, new_data), axis = 0)
    
    def normalize_Y(self):
        users = self.Y_data[:, 0] # all users - first col of the Y_data
        self.Ybar_data = self.Y_data.copy()
        self.mu = np.zeros((self.n_users,))
        for n in range(self.n_users):
            # row indices of rating done by user n
            # since indices need to be integers, we need to convert
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all ratings associated with user n
            item_ids = self.Y_data[ids, 1] 
            # and the corresponding ratings 
            ratings = self.Y_data[ids, 2]
            # take mean
            m = np.mean(ratings) 
            if np.isnan(m):
                m = 0 # to avoid empty array and nan value
            self.mu[n] = m
            # normalize
            self.Ybar_data[ids, 2] = ratings - self.mu[n]

        ################################################
        # form the rating matrix as a sparse matrix. Sparsity is important 
        # for both memory and computing efficiency. For example, if #user = 1M, 
        # #item = 100k, then shape of the rating matrix would be (100k, 1M), 
        # you may not have enough memory to store this. Then, instead, we store 
        # nonzeros only, and, of course, their locations.
        self.Ybar = sparse.coo_matrix((self.Ybar_data[:, 2],
            (self.Ybar_data[:, 1], self.Ybar_data[:, 0])), (self.n_items, self.n_users))
        self.Ybar = self.Ybar.tocsr()

    def similarity(self):
        eps = 1e-6
        self.S = self.dist_func(self.Ybar.T, self.Ybar.T)
    
        
    def refresh(self):
        """
        Normalize data and calculate similarity matrix again (after
        some few ratings added)
        """
        self.normalize_Y()
        self.similarity() 
        
    def fit(self):
        self.refresh()
        
    
    def __pred(self, u, i, normalized = 1):
        """ 
        predict the rating of user u for item i (normalized)
        if you need the un
        """
        # Step 1: find all users who rated i
        ids = np.where(self.Y_data[:, 1] == i)[0].astype(np.int32)
        # Step 2: 
        users_rated_i = (self.Y_data[ids, 0]).astype(np.int32)
        # Step 3: find similarity btw the current user and others 
        # who already rated i
        sim = self.S[u, users_rated_i]
        # Step 4: find the k most similarity users
        a = np.argsort(sim)[-self.k:] 
        # and the corresponding similarity levels
        nearest_s = sim[a]
        # How did each of 'near' users rated item i
        r = self.Ybar[i, users_rated_i[a]]
        if normalized:
            # add a small number, for instance, 1e-8, to avoid dividing by 0
            return (r*nearest_s)[0]/(np.abs(nearest_s).sum() + 1e-8)

        return (r*nearest_s)[0]/(np.abs(nearest_s).sum() + 1e-8) + self.mu[u]
    
    def pred(self, u, i, normalized = 1):
        """ 
        predict the rating of user u for item i (normalized)
        if you need the un
        """
        if self.uuCF: return self.__pred(u, i, normalized)
        return self.__pred(i, u, normalized)
            
    
    def recommend(self, u):
        """
        Determine all items should be recommended for user u.
        The decision is made based on all i such that:
        self.pred(u, i) > 0. Suppose we are considering items which 
        have not been rated by u yet. 
        """
        ids = np.where(self.Y_data[:, 0] == u)[0]
        items_rated_by_u = self.Y_data[ids, 1].tolist()              
        recommended_items = []
        for i in range(self.n_items):
            if i not in items_rated_by_u:
                rating = self.__pred(u, i)
                if rating > 0: 
                    recommended_items.append(i)
        
        return recommended_items 
    
    def recommend2(self, u):
        """
        Determine all items should be recommended for user u.
        The decision is made based on all i such that:
        self.pred(u, i) > 0. Suppose we are considering items which 
        have not been rated by u yet. 
        """
        ids = np.where(self.Y_data[:, 0] == u)[0]
        items_rated_by_u = self.Y_data[ids, 1].tolist()              
        recommended_items = []
    
        for i in range(self.n_items):
            if i not in items_rated_by_u:
                rating = self.__pred(u, i)
                if rating > 0: 
                    recommended_items.append(i)
        
        return recommended_items 

    def print_recommendation(self):
        """
        print all items which should be recommended for each user 
        """
        print ('Recommendation: ')
        for u in range(self.n_users):
            recommended_items = self.recommend(u)
            if self.uuCF:
                print ('    Recommend item(s):', recommended_items, 'for user', u)
            else: 
                print ('    Recommend item', u, 'for user(s) : ', recommended_items)
# rs = CF(Y_data, k = 2, uuCF = 1)
# rs.fit()

# rs.print_recommendation()
# 
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-100k/ub.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ub.test', sep='\t', names=r_cols, encoding='latin-1')

rate_train = ratings_base.to_numpy()
rate_test = ratings_test.to_numpy()

# indices start from 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1
rs = CF(rate_train, k = 2, uuCF = 1)
rs.fit()
rs.print_recommendation()

C:\Users\HuongDT2\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:54: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float


Recommendation: 
    Recommend item(s): [46, 63, 91, 227, 272, 275, 278, 284, 285, 286, 287, 292, 295, 296, 297, 301, 302, 305, 312, 314, 315, 316, 317, 319, 323, 330, 332, 339, 344, 345, 346, 351, 356, 380, 389, 402, 403, 407, 409, 411, 419, 420, 422, 424, 426, 427, 428, 429, 432, 434, 435, 442, 445, 446, 450, 461, 465, 466, 473, 474, 478, 479, 482, 486, 488, 489, 490, 492, 496, 497, 501, 503, 505, 506, 507, 510, 511, 512, 513, 516, 517, 519, 520, 521, 524, 525, 526, 530, 533, 537, 542, 545, 557, 558, 561, 567, 579, 580, 581, 588, 600, 602, 605, 607, 609, 612, 613, 614, 615, 616, 625, 629, 632, 637, 639, 640, 641, 644, 645, 646, 649, 650, 651, 652, 653, 654, 658, 660, 663, 671, 672, 674, 678, 685, 692, 693, 700, 701, 704, 708, 712, 714, 717, 723, 732, 734, 735, 739, 744, 745, 749, 752, 761, 762, 792, 810, 813, 834, 836, 844, 846, 847, 849, 850, 852, 853, 855, 864, 866, 867, 879, 886, 895, 905, 908, 914, 918, 921, 922, 926, 944, 952, 962, 964, 967, 970, 1004, 1009, 1011, 1018, 1019, 10

    Recommend item(s): [9, 45, 47, 54, 56, 59, 60, 72, 82, 90, 95, 98, 112, 113, 118, 121, 127, 171, 178, 189, 200, 206, 210, 248, 250, 252, 261, 272, 274, 282, 304, 312, 314, 318, 319, 344, 356, 358, 361, 377, 386, 396, 403, 419, 429, 430, 432, 442, 468, 477, 507, 542, 547, 557, 587, 591, 600, 602, 604, 606, 609, 610, 612, 615, 620, 632, 640, 641, 643, 644, 646, 647, 652, 653, 656, 662, 670, 675, 692, 693, 698, 704, 706, 708, 714, 716, 717, 735, 736, 752, 754, 792, 793, 796, 813, 834, 850, 852, 853, 862, 866, 867, 881, 895, 897, 899, 903, 904, 918, 921, 922, 951, 952, 954, 955, 957, 962, 964, 966, 970, 1006, 1015, 1021, 1025, 1038, 1054, 1059, 1076, 1079, 1083, 1084, 1092, 1102, 1109, 1110, 1118, 1125, 1126, 1136, 1142, 1143, 1151, 1156, 1158, 1159, 1166, 1175, 1188, 1191, 1192, 1198, 1202, 1232, 1239, 1261, 1267, 1268, 1285, 1292, 1297, 1301, 1306, 1307, 1331, 1343, 1346, 1353, 1357, 1366, 1367, 1376, 1385, 1388, 1390, 1393, 1397, 1404, 1417, 1425, 1427, 1428, 1431, 1432, 1448, 1449,

    Recommend item(s): [9, 15, 17, 18, 43, 46, 56, 80, 92, 100, 112, 113, 114, 118, 130, 132, 135, 139, 155, 161, 168, 170, 188, 191, 202, 206, 212, 213, 247, 249, 266, 296, 297, 389, 407, 433, 465, 469, 486, 489, 494, 495, 497, 512, 520, 527, 533, 542, 561, 573, 580, 581, 591, 592, 604, 610, 625, 632, 633, 640, 641, 643, 647, 648, 652, 657, 659, 675, 692, 698, 700, 706, 707, 712, 714, 717, 918, 922, 926, 954, 955, 960, 962, 964, 965, 970, 992, 1006, 1019, 1021, 1025, 1079, 1084, 1102, 1120, 1125, 1130, 1134, 1140, 1141, 1142, 1143, 1151, 1157, 1173, 1174, 1183, 1188, 1191, 1194, 1198, 1202, 1203, 1232, 1234, 1239, 1250, 1251, 1259, 1261, 1268, 1277, 1285, 1292, 1297, 1301, 1305, 1343, 1356, 1357, 1366, 1367, 1376, 1388, 1397, 1404, 1428, 1447, 1448, 1449, 1450, 1462, 1465, 1466, 1472, 1485, 1492, 1493, 1494, 1496, 1498, 1499, 1513, 1515, 1528, 1535, 1549, 1553, 1557, 1584, 1591, 1593, 1597, 1598, 1604, 1606, 1609, 1622, 1627, 1628, 1635, 1641, 1642, 1644, 1649, 1650, 1652, 1655, 1661]

    Recommend item(s): [4, 11, 18, 37, 49, 52, 54, 60, 61, 70, 72, 73, 82, 85, 86, 94, 99, 107, 108, 113, 115, 131, 132, 134, 135, 136, 142, 146, 149, 155, 160, 164, 170, 172, 177, 180, 182, 187, 188, 189, 191, 193, 194, 196, 198, 199, 201, 202, 204, 207, 211, 212, 218, 220, 221, 227, 228, 229, 241, 243, 245, 250, 252, 258, 262, 266, 276, 278, 291, 303, 315, 318, 319, 333, 345, 357, 361, 388, 389, 390, 398, 407, 415, 418, 421, 424, 428, 431, 433, 435, 439, 443, 445, 454, 464, 466, 477, 483, 486, 487, 496, 497, 503, 505, 509, 510, 512, 514, 516, 521, 523, 528, 532, 552, 557, 572, 573, 588, 591, 592, 600, 602, 603, 604, 606, 611, 613, 614, 615, 616, 620, 623, 624, 625, 631, 632, 634, 639, 645, 647, 650, 652, 658, 660, 666, 667, 670, 685, 696, 700, 708, 714, 717, 732, 735, 749, 759, 792, 793, 810, 812, 813, 834, 835, 847, 848, 850, 863, 866, 867, 873, 895, 897, 901, 908, 921, 922, 943, 952, 955, 957, 962, 967, 968, 976, 1018, 1020, 1021, 1023, 1045, 1051, 1078, 1084, 1108, 1124, 1142, 114

...

    Recommend item(s): [12, 25, 32, 38, 46, 47, 48, 72, 86, 90, 100, 115, 118, 120, 124, 132, 133, 134, 135, 138, 143, 151, 169, 176, 177, 178, 186, 189, 192, 193, 195, 204, 211, 213, 216, 222, 223, 235, 245, 247, 249, 252, 265, 269, 271, 274, 282, 297, 312, 318, 329, 330, 331, 333, 336, 344, 352, 366, 377, 389, 391, 415, 419, 424, 433, 447, 457, 463, 480, 483, 484, 485, 488, 492, 493, 495, 498, 500, 501, 502, 508, 510, 511, 512, 513, 516, 521, 522, 523, 525, 531, 533, 536, 547, 552, 556, 557, 572, 579, 583, 591, 592, 605, 610, 613, 616, 620, 625, 627, 631, 633, 640, 647, 651, 652, 653, 657, 659, 660, 662, 663, 666, 669, 685, 693, 694, 698, 704, 712, 714, 717, 720, 721, 729, 735, 736, 742, 745, 763, 777, 786, 788, 791, 792, 793, 799, 805, 813, 824, 832, 849, 852, 853, 855, 866, 879, 881, 883, 885, 887, 895, 899, 903, 904, 905, 920, 926, 935, 952, 953, 957, 960, 965, 966, 970, 978, 988, 992, 994, 1007, 1018, 1019, 1021, 1044, 1054, 1061, 1063, 1064, 1065, 1072, 1079, 1083, 1092, 1097, 1

    Recommend item(s): [5, 7, 10, 11, 13, 21, 31, 38, 47, 49, 55, 56, 59, 60, 63, 69, 70, 83, 90, 92, 94, 99, 100, 105, 114, 128, 131, 132, 135, 139, 142, 143, 149, 154, 163, 169, 175, 178, 180, 199, 204, 219, 227, 228, 229, 232, 235, 239, 242, 246, 250, 252, 261, 275, 277, 278, 279, 280, 284, 286, 291, 295, 302, 304, 308, 314, 315, 318, 319, 324, 344, 370, 379, 389, 402, 407, 420, 430, 465, 468, 470, 473, 474, 480, 482, 483, 484, 487, 489, 492, 500, 503, 510, 512, 522, 530, 533, 535, 557, 574, 590, 594, 603, 605, 606, 612, 614, 630, 632, 633, 640, 653, 656, 661, 664, 685, 689, 698, 707, 712, 729, 740, 743, 768, 780, 784, 792, 813, 819, 835, 844, 849, 850, 852, 853, 857, 864, 865, 866, 867, 870, 895, 898, 899, 901, 921, 927, 935, 936, 952, 955, 958, 964, 965, 970, 992, 1005, 1011, 1018, 1021, 1025, 1030, 1038, 1040, 1061, 1062, 1063, 1065, 1066, 1083, 1087, 1106, 1120, 1128, 1130, 1131, 1141, 1142, 1159, 1171, 1173, 1183, 1189, 1198, 1220, 1234, 1239, 1241, 1261, 1264, 1267, 1268, 1285

    Recommend item(s): [7, 11, 18, 21, 32, 44, 47, 49, 50, 54, 55, 56, 58, 59, 63, 86, 95, 96, 97, 98, 100, 113, 115, 118, 125, 131, 132, 133, 142, 149, 150, 166, 167, 168, 169, 170, 172, 174, 179, 180, 181, 184, 185, 191, 193, 195, 197, 198, 206, 207, 210, 215, 222, 223, 236, 237, 241, 245, 249, 250, 252, 256, 259, 261, 268, 271, 272, 275, 284, 291, 301, 308, 310, 317, 319, 325, 326, 328, 341, 343, 346, 370, 378, 380, 381, 384, 393, 401, 403, 407, 409, 418, 427, 428, 429, 430, 431, 433, 442, 450, 461, 463, 464, 465, 466, 473, 474, 482, 487, 488, 489, 490, 493, 495, 496, 510, 512, 516, 518, 519, 521, 524, 528, 537, 542, 565, 571, 581, 600, 602, 606, 608, 609, 632, 637, 640, 643, 644, 646, 647, 650, 653, 656, 657, 658, 659, 661, 662, 664, 677, 682, 683, 685, 692, 709, 717, 723, 741, 743, 748, 799, 812, 832, 833, 834, 837, 852, 857, 862, 865, 871, 879, 881, 882, 883, 887, 896, 897, 899, 901, 905, 935, 941, 944, 949, 957, 962, 965, 968, 1006, 1015, 1018, 1020, 1021, 1037, 1038, 1059, 1069

    Recommend item(s): [5, 9, 12, 18, 21, 22, 26, 27, 30, 41, 43, 45, 47, 49, 51, 57, 58, 59, 60, 67, 68, 82, 90, 91, 92, 123, 125, 126, 131, 136, 139, 142, 146, 149, 181, 183, 185, 186, 189, 191, 206, 213, 214, 215, 223, 226, 230, 232, 236, 243, 245, 248, 249, 250, 255, 256, 257, 261, 264, 266, 267, 268, 271, 274, 275, 277, 282, 285, 286, 295, 297, 301, 302, 304, 312, 314, 315, 317, 318, 319, 323, 332, 333, 339, 343, 344, 346, 361, 377, 386, 389, 407, 418, 424, 432, 463, 464, 465, 468, 484, 496, 499, 500, 504, 507, 511, 521, 533, 542, 547, 557, 572, 573, 579, 580, 581, 591, 592, 595, 600, 604, 609, 620, 625, 629, 630, 633, 640, 641, 643, 650, 652, 669, 678, 691, 692, 693, 696, 698, 706, 708, 712, 714, 717, 729, 731, 734, 736, 746, 750, 790, 792, 793, 813, 834, 850, 852, 853, 855, 866, 867, 873, 881, 885, 886, 897, 899, 903, 904, 908, 914, 918, 922, 924, 935, 951, 952, 954, 955, 957, 966, 970, 1004, 1021, 1038, 1062, 1063, 1079, 1084, 1100, 1102, 1103, 1110, 1124, 1130, 1136, 1137, 114

    Recommend item(s): [0, 4, 6, 7, 10, 14, 24, 25, 27, 47, 51, 52, 56, 58, 59, 60, 78, 81, 82, 90, 92, 97, 113, 114, 118, 134, 143, 148, 149, 164, 167, 169, 171, 172, 174, 177, 188, 189, 190, 191, 195, 196, 198, 199, 200, 201, 209, 212, 222, 225, 228, 229, 237, 240, 242, 248, 250, 252, 258, 264, 266, 271, 272, 287, 288, 297, 303, 305, 309, 310, 312, 314, 317, 326, 338, 343, 350, 352, 356, 358, 379, 384, 391, 402, 407, 418, 428, 431, 432, 434, 435, 443, 451, 453, 460, 463, 473, 474, 476, 482, 486, 489, 503, 509, 512, 513, 515, 516, 518, 535, 542, 547, 549, 564, 572, 580, 590, 591, 606, 620, 625, 644, 646, 647, 650, 655, 656, 658, 659, 660, 666, 667, 669, 674, 685, 688, 694, 697, 698, 717, 735, 747, 749, 750, 759, 764, 769, 770, 780, 792, 804, 805, 813, 824, 832, 839, 850, 852, 853, 857, 862, 866, 867, 871, 872, 879, 881, 882, 883, 884, 886, 888, 894, 895, 898, 899, 905, 908, 915, 918, 921, 923, 926, 946, 957, 960, 967, 968, 976, 992, 1004, 1021, 1023, 1024, 1064, 1079, 1083, 1084, 1100